# Cleaning Data: Homework

In class, we discussed a few different ways to clean data — simple Python methods, Pandas versions of those methods, and regular expressions (both in plain Python and in Pandas). You are welcome to use whatever combination of those approaches you'd like. If you really want to stretch your skills, try solving the questions a few different ways.

## 0) Load `pets.csv` (provided in CourseWorks)

... and assign the data to a variable named `pets`.

In [113]:
import pandas as pd
pets = pd.read_csv('pets.csv')
pets

,name,species,weight,years_old
0,"smith, fido",dog,5 lbs,1
1,"SMITH,, Bella",CAT,7 lbs,10
2,"Ortiz, Milo",dog,15 lbs,5
3,"Smith, Goldie",goldfish,7 oz,???
4,"Kim, Fuzz",cat,3 lbs,5 (five)
5,"Jones, Las Vegas",DOG,12 lbs,unknown
6,Jones; fifi,puppy,2 lbs,0.25
7,"Lee, nemo",gold fish,10 oz,2
8,"Chan, ZIGGY",kitten,2 lbs,0.5
9,"Chan, Zog",dog,20 lbs,7


We're going to clean the data so that you can __programmatically__ answer the following questions:

- Which family owns the most pets? (All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)
---
- How many of each kind of animal are there? (Puppies should count as dogs, and kittens as cats.)
---
- What is the total weight of the pets? (And which family's pets weigh the most, total?)
---
- Of the *known* ages, what's the average pet age?

## 1) Create a new column in the DataFrame called `last_name`, containing *just* the pet's surname

... normalized so that `.value_counts()` will count `smith, fido` and `SMITH, Bella` as being from the same family.

In [114]:
pets["last_name"] = (
    pets["name"]
    .str.replace(";",",")
    .str.split(",")
    .str.get(0)
    .str.title()
)
pets

,name,species,weight,years_old,last_name
0,"smith, fido",dog,5 lbs,1,Smith
1,"SMITH,, Bella",CAT,7 lbs,10,Smith
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz
3,"Smith, Goldie",goldfish,7 oz,???,Smith
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones
6,Jones; fifi,puppy,2 lbs,0.25,Jones
7,"Lee, nemo",gold fish,10 oz,2,Lee
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan
9,"Chan, Zog",dog,20 lbs,7,Chan


## 2) Check how many distinct last names you see

(There should be 6.)

In [115]:
counts = pets["last_name"].nunique()
print(counts)

6


## 3) Calculate which family owns the most pets

(All pets have been given the surname of the family that owns them. Assume that there's only one family with each surname.)

In [116]:
counts = pets["last_name"].value_counts()
print(counts)
pets

last_name
Smith    3
Jones    2
Chan     2
Ortiz    1
Kim      1
Lee      1
Name: count, dtype: int64


,name,species,weight,years_old,last_name
0,"smith, fido",dog,5 lbs,1,Smith
1,"SMITH,, Bella",CAT,7 lbs,10,Smith
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz
3,"Smith, Goldie",goldfish,7 oz,???,Smith
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones
6,Jones; fifi,puppy,2 lbs,0.25,Jones
7,"Lee, nemo",gold fish,10 oz,2,Lee
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan
9,"Chan, Zog",dog,20 lbs,7,Chan


## 4) Create a new column called `animal_type`, which standardizes the `species` column

(Puppies should count as dogs, and kittens as cats; and there should only be one spelling of `gold fish`/`goldfish`.)

In [117]:
pets["animal_type"] = (
    pets["species"]
    .str.lower()
    .str.replace("puppy","dog") #doing the .str before makes it only work on the column, not the whole table
    .str.replace("kitten","dog")
    .str.replace("gold fish","goldfish")
)

pets

,name,species,weight,years_old,last_name,animal_type
0,"smith, fido",dog,5 lbs,1,Smith,dog
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,dog
9,"Chan, Zog",dog,20 lbs,7,Chan,dog


## 5) How many of each kind of animal are there?

In [118]:
species_counts = pets["animal_type"].value_counts()
print(species_counts)

animal_type
dog         6
cat         2
goldfish    2
Name: count, dtype: int64


## 6) Create a new column called `weight_quantity`, which represents the numeric component of the `weight` column, as an integer

(i.e., `"10 oz"` -> `10`)

In [119]:
pets["weight_quantity"] = (
    pets["weight"]
    .str.extract(r"^(\d+)", expand=False)
)
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity
0,"smith, fido",dog,5 lbs,1,Smith,dog,5
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,dog,2
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20


## 7) Create a new column called `weight_unit`, which represents the units component of the `weight` column

(i.e., `"10 oz"` -> `"oz"`)

In [120]:
pets["weight_unit"] = (
    pets["weight"]
    .str.split(" ")
    .str.get(1)

)

pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit
0,"smith, fido",dog,5 lbs,1,Smith,dog,5,lbs
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7,lbs
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15,lbs
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7,oz
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3,lbs
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12,lbs
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2,lbs
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10,oz
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,dog,2,lbs
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20,lbs


## 8) Create a new column called `unit_factor`, which should equal `1` when `weight_unit` is `"lbs"` and `0.0625` when `weight_unit` is `"oz"`

(i.e., `"10 oz"` -> `"oz"`)

You could do this by defining a function and using `column.apply(my_function)` or through Pandas' `.replace({ ... })` method, which we didn't cover in class but is somewhat easier here. That would look like this (yes, I'm giving you the answer here):

```python
pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
)
```

In [121]:
pets["weight_quantity"] = (
    pets["weight_quantity"]
    .astype(float)
)

pets["unit_factor"] = (
    pets["weight_unit"]
    .replace({
        "lbs": 1,
        "oz": 0.0625
    })
    .astype(float)
)

pets

/var/folders/qn/5c47rdh16nb5dtrhgstp_1cr0000gp/T/ipykernel_67842/945135568.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace({


,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit,unit_factor
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0,lbs,1.0000
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0,lbs,1.0000
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0,lbs,1.0000
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0,oz,0.0625
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0,lbs,1.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0,lbs,1.0000
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0,lbs,1.0000
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0,oz,0.0625
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,dog,2.0,lbs,1.0000
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0,lbs,1.0000


## 8) Create a new column called `weight_lbs`, which calculates the pet's weight in lbs by multiplying `weight_quantity` by `unit_factor`

In [122]:
pets["weight_lbs"] = pets["weight_quantity"] * pets["unit_factor"]
pets

,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit,unit_factor,weight_lbs
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0,lbs,1.0000,5.0000
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0,lbs,1.0000,7.0000
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0,lbs,1.0000,15.0000
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0,oz,0.0625,0.4375
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0,lbs,1.0000,3.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0,lbs,1.0000,12.0000
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0,lbs,1.0000,2.0000
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0,oz,0.0625,0.6250
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,dog,2.0,lbs,1.0000,2.0000
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0,lbs,1.0000,20.0000


## 9) What is the total weight of the pets?

In [123]:
total_weight = sum(pets["weight_lbs"])
print(total_weight)

TypeError: 'str' object is not callable

## 10)  And which family's pets weigh the most, total?

In [ ]:
pets = pets.sort_values('weight_lbs', ascending=True)
pets


,name,species,weight,years_old,last_name,animal_type,weight_quantity,weight_unit,unit_factor,weight_lbs
3,"Smith, Goldie",goldfish,7 oz,???,Smith,goldfish,7.0,oz,0.0625,0.4375
7,"Lee, nemo",gold fish,10 oz,2,Lee,goldfish,10.0,oz,0.0625,0.6250
6,Jones; fifi,puppy,2 lbs,0.25,Jones,dog,2.0,lbs,1.0000,2.0000
8,"Chan, ZIGGY",kitten,2 lbs,0.5,Chan,dog,2.0,lbs,1.0000,2.0000
4,"Kim, Fuzz",cat,3 lbs,5 (five),Kim,cat,3.0,lbs,1.0000,3.0000
0,"smith, fido",dog,5 lbs,1,Smith,dog,5.0,lbs,1.0000,5.0000
1,"SMITH,, Bella",CAT,7 lbs,10,Smith,cat,7.0,lbs,1.0000,7.0000
5,"Jones, Las Vegas",DOG,12 lbs,unknown,Jones,dog,12.0,lbs,1.0000,12.0000
2,"Ortiz, Milo",dog,15 lbs,5,Ortiz,dog,15.0,lbs,1.0000,15.0000
9,"Chan, Zog",dog,20 lbs,7,Chan,dog,20.0,lbs,1.0000,20.0000


## 11) Of the known ages, what's the average pet age?

(You should get `3.84375`; if not, make sure you're accounting for numbers like `0.5` correctly.)

In [ ]:
pets["tally"] = (
    pets["years_old"]
    .str.replace("???", "NaN")
    .str.replace(" (five)", "")
    .str.replace("unknown", "NaN")
)
# pets = pets.sort_values("tally", ascending=True)
sum = pets["tally"].sum()
print(sum)

# pets.to_csv("pet_names", index=False)
#you can also do without the index part, but that deletes the numbers on the left hand side, which can start
# to add up when you pass documents between people repeatedly

pd.DataFrame(pets["last_name"]["weight_unit"])



pets

1 105NaN5NaN0.2520.57


KeyError: 'weight_unit'

---

---

---